<a href="https://colab.research.google.com/github/laurenx1/learning-systems/blob/main/ps8_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# imports
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

from sklearn.linear_model import Perceptron
import random
import math
from typing import List
from itertools import product
import scipy.special
from scipy import optimize
import scipy.optimize as spo
from sympy import Symbol, Derivative
import functools
from sklearn import svm, model_selection
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold


# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# loading the data
train = np.loadtxt('/content/drive/MyDrive/features.train')
test = np.loadtxt('/content/drive/MyDrive/features.test')

X_train = train[:,1:]
Y_train = train[:,0]
N_train = X_train[:, 0].size

X_test = test[:,1:]
Y_test = test[:,0]
N_test = X_test[:, 0].size

In [ ]:
C = .01
Q = 2

In [ ]:
def to_binary(Y, classes):
    return np.array([1 if y == classes else -1 for y in Y])

def score(y_class):
    # Set all other labels to 0 to make it a binary y_class vs all classification
    bin_Y_train = to_binary(Y_train, y_class)
    bin_Y_test = to_binary(Y_test, y_class)

    clf = svm.SVC(kernel='poly', C=C, degree=Q, gamma=1.0, coef0=1.0, cache_size=20000)
    clf.fit(X_train, bin_Y_train)
    print(f"Score of {y_class} versus all = {clf.score(X_test, bin_Y_test)}")

In [ ]:
for i in range(10):
  score(i)

Score of 0 versus all = 0.8883906327852517
Score of 1 versus all = 0.9780767314399601
Score of 2 versus all = 0.9013452914798207
Score of 3 versus all = 0.9172894867962132
Score of 4 versus all = 0.9003487792725461
Score of 5 versus all = 0.9202790234180369
Score of 6 versus all = 0.9152964623816642
Score of 7 versus all = 0.9267563527653214
Score of 8 versus all = 0.9172894867962132
Score of 9 versus all = 0.9118086696562033


In [ ]:
def num_support_vectors(y_class):
    # Set all other labels to 0 to make it a binary y_class vs all classification
    bin_Y_train = to_binary(Y_train, y_class)
    bin_Y_test = to_binary(Y_test, y_class)

    clf = svm.SVC(kernel='poly', C=C, degree=Q, gamma=1.0, coef0=1.0, cache_size=20000)
    clf.fit(X_train, bin_Y_train)
    return sum(clf.n_support_)

In [ ]:
num_support_vectors(0) - num_support_vectors(1)

In [ ]:
def run_svm_tuning(X_train, Y_train, X_test, Y_test, q_value=2, num_runs=100, C_values=[.0001, .001, .01, .1, 1]):
    results = []
    cross_val_errors = []

    def get_binary_data(data, first_class, second_class):
        X, Y = data
        indices = (Y == first_class) | (Y == second_class)
        return X[indices], Y[indices]

    X_test_1vs5, Y_test_1vs5 = get_binary_data((X_test, Y_test), 1, 5)
    X_train_1vs5, Y_train_1vs5 = get_binary_data((X_train, Y_train), 1, 5)

    for _ in range(num_runs):
        for train_idx, val_idx in zip(range(0, len(X_train_1vs5), len(X_train_1vs5)//10),
                                      range(len(X_train_1vs5)//10, len(X_train_1vs5)+1, len(X_train_1vs5)//10)):
            X_train_subset, X_val = X_train_1vs5[:train_idx] + X_train_1vs5[val_idx:], X_train_1vs5[train_idx:val_idx]
            Y_train_subset, Y_val = Y_train_1vs5[:train_idx] + Y_train_1vs5[val_idx:], Y_train_1vs5[train_idx:val_idx]

            top_score = 0
            best_parameters = None
            for C_value in C_values:
                for q_value in range(q_value):  # Changed variable name from Qs to q_value
                    clf = svm.SVC(kernel='poly', C=C_value, degree=q_value, gamma=1.0, coef0=1.0, cache_size=20000)
                    clf.fit(X_train_subset, Y_train_subset)
                    current_score = clf.score(X_val, Y_val)
                    if current_score > top_score:
                        top_score = current_score
                        best_parameters = (C_value, q_value)

            results.append(best_parameters)
            clf = svm.SVC(kernel='poly', C=best_parameters[0], degree=best_parameters[1], gamma=1.0, coef0=1.0, cache_size=20000)
            clf.fit(X_train_subset, Y_train_subset)
            cross_val_errors.append(1 - clf.score(X_val, Y_val))

    unique_parameters, counts = np.unique(results, return_counts=True, axis=0).T
    print("Unique Parameters: ", unique_parameters)
    print("Counts: ", counts)
    average_error = np.mean(cross_val_errors)
    print("Average Cross-Validation Error: ", average_error)

In [ ]:
def get_binary_data(data, first_class, second_class):
        X, Y = data
        indices = (Y == first_class) | (Y == second_class)
        return X[indices], Y[indices]

Cs = [0.01, 1, 100, 1e4, 1e6]

X_test_new,Y_test_new = get_binary_data((X_test, Y_test), 1, 5)
X_train_new, Y_train_new = get_binary_data((X_train, Y_train), 1, 5)

def compute_eIN(C):
    clf = svm.SVC(kernel='rbf', C=C, degree=Q, gamma=1.0, cache_size=20000)
    clf.fit(X_train_new, Y_train_new)
    return 1 - clf.score(X_train_new, Y_train_new)

print([compute_eIN(C) for C in Cs])

In [ ]:

def compute_eOUT(C):
    clf = svm.SVC(kernel='rbf', C=C, degree=Q, gamma=1.0, cache_size=20000)
    clf.fit(X_train_new, Y_train_new)
    return 1 - clf.score(X_test_new, Y_test_new)

print([compute_eOUT(C) for C in Cs])